In [2]:
#se importan las librerias
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import glob
from sklearn.svm import SVC
import seaborn as sns
from skimage.feature import hog
from skimage.transform import resize
import skimage.io as io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
#Se obtienen las direcciones del sistema para el workspace actual
current_directory = os.getcwd()
# se cargan las rutas de las imagenes de entrenamiento y validación
train_paths = glob.glob(os.path.join(current_directory,'train','*.jpg'))
valid_paths = glob.glob(os.path.join(current_directory,'val','*.jpg'))

In [4]:
# se construye el vector de etiquetas para las imagenes de entrenamiento y valicion
# 0 means dafodil
# 1 means snowdrop
# 2 means sunflower
# 3 means tigerlily
labels_train = [0]*5 + [1]*5 + [2]*5 + [3]*5 
labels_valid= [0]*5 + [1]*5 + [2]*5 + [3]*5 

In [5]:
# se leen las imagenes alojadas en las rutas especificadas
imagenes_train = []
for file_train in train_paths:
    img_train = io.imread(file_train)
    image_rescaled_train = resize(img_train, (150,150), anti_aliasing=False)# se reduce el tamaño de las imagenes
    imagenes_train.append(image_rescaled_train)

imagenes_valid = []
for file_valid in valid_paths:
    img_valid = plt.imread(file_valid)
    image_rescaled_valid = resize(img_valid, (150,150), anti_aliasing=False)# se reduce el tamaño de las imagenes
    imagenes_valid.append(image_rescaled_valid)

In [6]:
%run JointColorHistogram.py
""" Joint histogram for color images
    code by Maria Fernanda Roa
    mf.roa@uniandes.edu.co
    IBIO 3470 - Uniandes 
"""
import numpy as np
from skimage.util import img_as_float

def JointColorHistogram(img, num_bins, min_val=None, max_val=None):
    """Calculate joint histogram for color images

    Arguments:
        img (numpy.array) -- 2D color image 
        num_bins (array like of ints) -- Number of bins per channel. 
                                         If an int is given, all channels
                                         will have same ammount of bins.

    Keyword Arguments:
        min_val (array like of ints) -- Minimum intensity range value per channel
                                        If an int is given, all channels
                                        will have same minimmum. (default: {None})
        max_val (array like of ints) -- Maximum intensity range value per channel
                                        If an int is given, all channels
                                        will have same maximum. (default: {None})

    Returns:
        [numpy.array] -- Array containing joint color histogram of size num_bins.
    """   

    assert len(img.shape) == 3, 'img must be a color 2D image'
    #Transform image to float dtype 
    img = img_as_float(img)
    _, _, n_channels = img.shape

    #Verify input parameters
    assert isinstance(num_bins, (int, tuple, list, np.array)),'num_bins must be int or array like'
    if isinstance(num_bins, int):
        num_bins = np.array([num_bins]*n_channels)
    assert len(num_bins) == n_channels,'num_bins length and number of channels differ'
 
    if min_val is None:
        min_val = np.min(img, (0,1))
    else:
        assert isinstance(min_val, (int, tuple, list, np.array)),'min_val must be int or array like'
        if isinstance(min_val, int):
            min_val = np.array([min_val]*n_channels)
        else: 
            min_val = np.array(min_val)
    assert len(min_val) == n_channels,'min_val length and number of channels differ'
    min_val = min_val.reshape((1, 1, -1))

    if max_val is None:
        max_val = np.max(img, (0,1))
    else:
        assert isinstance(max_val, (int, tuple, list, np.array)),'max_val must be int or array like'
        if isinstance(max_val, int):
            max_val = np.array([max_val]*n_channels)
        else: 
            max_val = np.array(max_val)
    assert len(max_val) == n_channels,'max_val length and number of channels differ'
    max_val = max_val.reshape((1, 1, -1)) + 1e-5

    joint_hist = np.zeros(num_bins, dtype=np.int)
    num_bins = num_bins.reshape((1, 1, -1))

    # Scale intensities (intensities are scaled within the range for each channel)
    # Values now are between 0 and 1
    img = (img - min_val) / (max_val - min_val)
    # Calculate index matrix 
    idx_matrix = np.floor(img*num_bins).astype('int') 
    idx_matrix = idx_matrix.reshape((-1, n_channels))
    #Create joint histogram
    for p in range(len(idx_matrix)):
        joint_hist[tuple(idx_matrix[p, :])] += 1

    return joint_hist

## Punto 1

In [7]:
# se obtienen los histogramas conjuntos de las imagenes de entranamiento y validacion
hist1_train = []
hist2_train = []
hist3_train = []
for img_train in imagenes_train:
    hist1_train.append(JointColorHistogram(img_train, 100).ravel())# para 100 bins
    hist2_train.append(JointColorHistogram(img_train, 200).ravel())# para 200 bins
    hist3_train.append(JointColorHistogram(img_train, 255).ravel())# para 255 bins

hist1_train = np.array(hist1_train)    
hist2_train = np.array(hist2_train)  
hist3_train = np.array(hist3_train)  

hist1_valid = []
hist2_valid = []
hist3_valid = []
for img_valid in imagenes_valid:
    hist1_valid.append(JointColorHistogram(img_valid, 100).ravel())# para 100 bins
    hist2_valid.append(JointColorHistogram(img_valid, 200).ravel())# para 200 bins
    hist3_valid.append(JointColorHistogram(img_valid, 255).ravel())# para 255 bins
    
hist1_valid = np.array(hist1_valid)    
hist2_valid = np.array(hist2_valid)  
hist3_valid = np.array(hist3_valid)  

## Punto 1. SVM

In [11]:
sv_linear_hist1 = SVC(kernel='linear', verbose= False)
sv_linear_hist1.fit(hist1_train,labels_train)
labels_predict1 = sv_linear_hist1.predict(hist1_valid)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(sv_linear_hist1.score(hist1_valid, labels_valid)))

Accuracy 0.65 to 100 bins


In [12]:
sv_linear_hist2 = SVC(kernel='linear', verbose= False)
sv_linear_hist2.fit(hist2_train,labels_train)
labels_predict2 = sv_linear_hist2.predict(hist2_valid)
# Print the tuned parameters and score
print("Accuracy {} to 200 bins".format(sv_linear_hist2.score(hist2_valid, labels_valid)))

Accuracy 0.6 to 200 bins


In [15]:
sv_linear_hist3 = SVC(kernel='linear', verbose= False)
sv_linear_hist3.fit(hist3_train,labels_train)
labels_predict3 = sv_linear_hist3.predict(hist3_valid)
# Print the tuned parameters and score
print("Accuracy {} to 255 bins".format(sv_linear_hist3.score(hist3_valid, labels_valid)))

Accuracy 0.6 to 255 bins


## Punto 1. Random forest

In [16]:
rf1 = RandomForestClassifier(verbose= False)
rf1.fit(hist1_train,labels_train)
labels_predict1_rf = rf1.predict(hist1_valid)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(rf1.score(hist1_valid, labels_valid)))

Accuracy 0.35 to 100 bins


In [17]:
rf2 = RandomForestClassifier(verbose= False)
rf2.fit(hist2_train,labels_train)
labels_predict2_rf = rf2.predict(hist2_valid)
# Print the tuned parameters and score
print("Accuracy {} to 200 bins".format(rf2.score(hist2_valid, labels_valid)))

Accuracy 0.4 to 200 bins


In [18]:
rf3 = RandomForestClassifier(verbose= False)
rf3.fit(hist3_train,labels_train)
labels_predict3_rf = rf3.predict(hist3_valid)
# Print the tuned parameters and score
print("Accuracy {} to 255 bins".format(rf3.score(hist3_valid, labels_valid)))

Accuracy 0.5 to 255 bins


## Punto 2

In [12]:
# se obtienen los hog de las imagenes de entrenamiento y validacion
hist1_train_hog = []
hist2_train_hog = []
hist3_train_hog = []
hist4_train_hog = []
for img_train in imagenes_train:
    hist1_train_hog.append(hog(img_train, 8,feature_vector = True))# con 8 bins para orientaciones
    hist2_train_hog.append(hog(img_train, 24, feature_vector = True))# con 24 bins para orientaciones
    hist3_train_hog.append(hog(img_train, 8, cells_per_block=(7,7),feature_vector = True))# con 8 bins para orientaciones y 7 bloques
    hist4_train_hog.append(hog(img_train, 8, cells_per_block=(14,14),feature_vector = True))# con 8 bins para orientaciones y 14 bloques

hist1_train_hog = np.array(hist1_train_hog)    
hist2_train_hog = np.array(hist2_train_hog)  
hist3_train_hog = np.array(hist3_train_hog)  
hist4_train_hog = np.array(hist4_train_hog)

hist1_valid_hog = []
hist2_valid_hog = []
hist3_valid_hog = []
hist4_valid_hog = []
for img_valid in imagenes_valid:
    hist1_valid_hog.append(hog(img_valid, 8,feature_vector = True))# con 8 bins para orientaciones
    hist2_valid_hog.append(hog(img_valid, 24, feature_vector = True))# con 24 bins para orientaciones
    hist3_valid_hog.append(hog(img_valid, 8, cells_per_block=(7,7),feature_vector = True))# con 8 bins para orientaciones y 7 bloques
    hist4_valid_hog.append(hog(img_valid, 8, cells_per_block=(14,14),feature_vector = True))# con 8 bins para orientaciones y 14 bloques
    
hist1_valid_hog = np.array(hist1_valid_hog)    
hist2_valid_hog = np.array(hist2_valid_hog)  
hist3_valid_hog = np.array(hist3_valid_hog)  
hist4_valid_hog = np.array(hist4_valid_hog)

## Punto 2. SVM

In [14]:
sv_linear_hist1_HOG = SVC(kernel='linear', verbose= False)
sv_linear_hist1_HOG.fit(hist1_train_hog,labels_train)
labels_predict1_svm_hog = sv_linear_hist1_HOG.predict(hist1_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(sv_linear_hist1_HOG.score(hist1_valid_hog, labels_valid)))

Accuracy 0.75 to 100 bins


In [15]:
sv_linear_hist2_HOG = SVC(kernel='linear', verbose= False)
sv_linear_hist2_HOG.fit(hist2_train_hog,labels_train)
labels_predict2_svm_hog = sv_linear_hist2_HOG.predict(hist2_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(sv_linear_hist2_HOG.score(hist2_valid_hog, labels_valid)))

Accuracy 0.75 to 100 bins


In [16]:
sv_linear_hist3_HOG = SVC(kernel='linear', verbose= False)
sv_linear_hist3_HOG.fit(hist3_train_hog,labels_train)
labels_predict3_svm_hog = sv_linear_hist3_HOG.predict(hist3_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(sv_linear_hist3_HOG.score(hist3_valid_hog, labels_valid)))

Accuracy 0.8 to 100 bins


In [18]:
sv_linear_hist4_HOG = SVC(kernel='linear', verbose= False)
sv_linear_hist4_HOG.fit(hist4_train_hog,labels_train)
labels_predict4_svm_hog = sv_linear_hist4_HOG.predict(hist4_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(sv_linear_hist4_HOG.score(hist4_valid_hog, labels_valid)))

Accuracy 0.8 to 100 bins


## Punto 2. Random forest

In [21]:
rf1_hog = RandomForestClassifier(verbose= False)
rf1_hog.fit(hist1_train_hog,labels_train)
labels_predict1_rf_hog = rf1_hog.predict(hist1_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(rf1_hog.score(hist1_valid_hog, labels_valid)))

Accuracy 0.65 to 100 bins


In [22]:
rf2_hog = RandomForestClassifier(verbose= False)
rf2_hog.fit(hist2_train_hog,labels_train)
labels_predict2_rf_hog = rf2_hog.predict(hist2_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(rf2_hog.score(hist2_valid_hog, labels_valid)))

Accuracy 0.6 to 100 bins


In [23]:
rf3_hog = RandomForestClassifier(verbose= False)
rf3_hog.fit(hist3_train_hog,labels_train)
labels_predict3_rf_hog = rf3_hog.predict(hist3_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(rf3_hog.score(hist3_valid_hog, labels_valid)))

Accuracy 0.8 to 100 bins


In [24]:
rf4_hog = RandomForestClassifier(verbose= False)
rf4_hog.fit(hist4_train_hog,labels_train)
labels_predict4_rf_hog = rf4_hog.predict(hist4_valid_hog)
# Print the tuned parameters and score
print("Accuracy {} to 100 bins".format(rf4_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.75 to 100 bins


## Punto 3. SVM

In [ ]:
# modelos de SVM variando el kernel

In [8]:
sv_rbf1_hist = SVC(kernel='rbf', verbose= False)
sv_rbf1_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con RBF kernel en histograma conjunto de 100 bins".format(sv_rbf1_hist.score(hist1_valid, labels_valid)))

Accuracy 0.35 con RBF kernel en histograma conjunto de 100 bins


In [9]:
sv_rbf1_hog = SVC(kernel='rbf', verbose= False)
sv_rbf1_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con RBF kernel en hog de 8 bins".format(sv_rbf1_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.85 con RBF kernel en hog de 8 bins


In [10]:
sv_linear1_hist = SVC(kernel='linear', verbose= False)
sv_linear1_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con linear kernel en histograma conjunto de 100 bins".format(sv_linear1_hist.score(hist1_valid, labels_valid)))

Accuracy 0.65 con linear kernel en histograma conjunto de 100 bins


In [12]:
sv_linear1_hog = SVC(kernel='linear', verbose= False)
sv_linear1_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con linear kernel en hog de 8 bins".format(sv_linear1_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.8 con linear kernel en hog de 8 bins


In [ ]:
# modelos de SVM variando el la regularizacion

In [14]:
sv_c10_hist = SVC(kernel='rbf', C= 10, verbose= False)
sv_c10_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con regularizacion de 1/10 en histograma conjunto de 100 bins".format(sv_c10_hist.score(hist1_valid, labels_valid)))

Accuracy 0.55 con regularizacion de 1/10 en histograma conjunto de 100 bins


In [17]:
sv_c10_hog = SVC(kernel='rbf', C= 10, verbose= False)
sv_c10_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con regularizacion de 1/10 en hog de 8 bins".format(sv_c10_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.9 con regularizacion de 1/10 en hog de 8 bins


In [16]:
sv_c01_hist = SVC(kernel='rbf', C= 0.1, verbose= False)
sv_c01_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con regularizacion de 10 en histograma conjunto de 100 bins".format(sv_c01_hist.score(hist1_valid, labels_valid)))

Accuracy 0.35 con regularizacion de 10 en histograma conjunto de 100 bins


In [19]:
sv_c01_hog = SVC(kernel='rbf', C= 0.1, verbose= False)
sv_c01_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con regularizacion de 10 en hog de 8 bins".format(sv_c01_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.85 con regularizacion de 10 en hog de 8 bins


## Punto 4

In [ ]:
# modelos de RF variando The number of trees in the forest

In [11]:
rf1_1000Trees_hist = RandomForestClassifier(n_estimators=1000, verbose= False)
rf1_1000Trees_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con 1000 arboles de decision en histograma conjunto de 100 bins".format(rf1_1000Trees_hist.score(hist1_valid, labels_valid)))

Accuracy 0.4 con 1000 arboles de decision en histograma conjunto de 100 bins


In [14]:
rf1_1000Trees_hog = RandomForestClassifier(n_estimators=1000,verbose= False)
rf1_1000Trees_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con 1000 arboles de decision en hog de 8 bins".format(rf1_1000Trees_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.85 con 1000 arboles de decision en hog de 8 bins


In [15]:
rf1_10Trees_hist = RandomForestClassifier(n_estimators=10, verbose= False)
rf1_10Trees_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con 10 arboles de decision en histograma conjunto de 100 bins".format(rf1_10Trees_hist.score(hist1_valid, labels_valid)))

Accuracy 0.4 con 10 arboles de decision en histograma conjunto de 100 bins


In [16]:
rf1_10Trees_hog = RandomForestClassifier(n_estimators=1000,verbose= False)
rf1_10Trees_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con 10 arboles de decision en hog de 8 bins".format(rf1_10Trees_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.85 con 10 arboles de decision en hog de 8 bins


In [ ]:
# modelos de RF variando si hay o no boostraping

In [17]:
rf1_sinBT_hist = RandomForestClassifier(n_estimators=1000,bootstrap = False, verbose= False)
rf1_sinBT_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} sin bootstrap en histograma conjunto de 100 bins".format(rf1_sinBT_hist.score(hist1_valid, labels_valid)))

Accuracy 0.4 sin bootstrap en histograma conjunto de 100 bins


In [18]:
rf1_sinBT_hog = RandomForestClassifier(n_estimators=1000,bootstrap = False, verbose= False)
rf1_sinBT_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} sin bootstrap en hog de 8 bins".format(rf1_sinBT_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.75 sin bootstrap en hog de 8 bins


In [19]:
rf1_conBT_hist = RandomForestClassifier(n_estimators=1000,bootstrap = True, verbose= False)
rf1_conBT_hist.fit(hist1_train,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con bootstrap en histograma conjunto de 100 bins".format(rf1_conBT_hist.score(hist1_valid, labels_valid)))

Accuracy 0.45 con bootstrap en histograma conjunto de 100 bins


In [20]:
rf1_conBT_hog = RandomForestClassifier(n_estimators=1000,bootstrap = True, verbose= False)
rf1_conBT_hog.fit(hist4_train_hog,labels_train)
# Print the tuned parameters and score
print("Accuracy {} con bootstrap en hog de 8 bins".format(rf1_conBT_hog.score(hist4_valid_hog, labels_valid)))

Accuracy 0.9 con bootstrap en hog de 8 bins
